In [1]:
#Functions
# Extract text with PyPDF2
import PyPDF2
from PyPDF2 import PdfReader,PdfWriter
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
import pdfplumber
import fitz
pdf_name='PMBOK6-2017.pdf'

def extract_text_from_pdf_PyPDF2(pdf_name,debut=0,end=0):
    text=''
    pdf_file=open(pdf_name,'rb')
    pdf=PyPDF2.PdfReader(pdf_file)
    if end==0:
        end=len(pdf.pages)-1
    for pg_num in range(debut,end+1):
        page=pdf.pages[pg_num]
        text += page.extract_text ()
    return  text   
# Extract selected pages as pdf
def get_selected_pages_as_pdf(pdf_path,start,end):
    with open(pdf_path,'rb') as f:
        all_pdf=PdfReader(f)
        write_pdf=PdfWriter()
        new_pdf_name=f"{pdf_path}_from_page_{start}_{end}.pdf"
        if end!=len(all_pdf.pages):
            end+=1
        
        for page_num in range(start-1,end-1):
            pages=all_pdf.pages[page_num]
            write_pdf.add_page(pages)
        with open (new_pdf_name,'wb') as out:
            write_pdf.write(out)    

# Extract text with  pdfminer
def extract_text_from_pdf_pdfminer(pdf_path):
    laparams = LAParams()
    text = extract_text(pdf_path,laparams=laparams)
    return text

def organaize_title_with_sub_title(liste_titre,liste_titre_sub):
    dic_titre_position={}
    dic_titre={}
    liste_titre_position=[]
    liste_only_titre=[]
    for index1 in range(len(liste_titre)) :
        for index in range(len(liste_titre_sub)):
            space=liste_titre[index1][0].find(' ')
            if liste_titre_sub[index][0].find(liste_titre[index1][0][:space])>=0:
                liste_titre_position.append(liste_titre_sub[index])
                
                liste_only_titre.append(liste_titre_sub[index][0])
        
        dic_titre_position[liste_titre[index1]]=liste_titre_position
        
        dic_titre[liste_titre[index1][0]]=liste_only_titre
        
        liste_titre_position=[]
        
        liste_only_titre=[]
        
    return dic_titre,dic_titre_position
def sort_liste_based_on_element(liste,element):
        sorted_list=sorted(liste,key=lambda x:x[element])
        return sorted_list
    

def get_liste_of_slice(dic_position_liste,last_end=-1): #require a dictionary that contains the position of each title
    dic_contenu={}
    liste=[]
    dic_position_liste=list(dic_position_liste)
    dic_position_liste=sort_liste_based_on_element(dic_position_liste,1)#sort liste_element based on second element example ('11.1',102)>('11.2',106)
    for i in range(len(dic_position_liste)):
        start_position=int(dic_position_liste[i][1])
        if i!=len(dic_position_liste)-1:
            end_position=int(dic_position_liste[i+1][1])
        else:
            end_position=last_end
        liste.append([start_position,end_position])
    
         
        
    return liste





def get_contenu(dic_position,text): # require a dictionary that contains the position of each title
    dic_contenu={}
    liste_slice=get_liste_of_slice(dic_position)
    print(liste_slice)
    for i in range(len(liste_slice)):
        start_position=liste_slice[i][0]
        end_position=liste_slice[i][1]
        dic_contenu[list(dic_position.keys())[i][0]]=text[start_position:end_position]
    return dic_contenu

def get_detailed_contenu(dic_position_title,text): # require same as get_contenu , but it dig deeper and get content of sub_titles
    
    liste_of_keys=list(dic_position_title.keys())
    dic_contenu={}
    
    for index in range(len(liste_of_keys)):
        
        liste_value_per_key=dic_position_title[liste_of_keys[index]]
        if index <=len(liste_of_keys)-2:
            last_position=liste_of_keys[index+1][1]
        else:
            last_position=-1
        liste_slice_of_value=get_liste_of_slice(liste_value_per_key,last_position)
            
        for i in range(len(liste_slice_of_value)):
            start_position=liste_slice_of_value[i][0]
            end_position=liste_slice_of_value[i][1]
            dic_contenu[liste_value_per_key[i][0]]=text[start_position:end_position]
    return dic_contenu    
    
    
    
# Extract tables using pdfplumber

def extract_tables_pdfplumber(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_number in range(len(pdf.pages)):
            page = pdf.pages[page_number]
            page_tables = page.extract_tables()
            if page_tables:
                for table in page_tables:
                    tables.append(table)
    
    return tables
    return tables    
    
    
def extract_text_from_pdf_usingfitz(pdf_path):
    doc = fitz.open(pdf_path)
    text = ''

    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    doc.close()
    return text



In [2]:
intro_name='PMBOK6-2017.pdf_from_page_431_436.pdf'
chapiter_name='PMBOK6-2017.pdf_from_page_437_494.pdf'

In [3]:
extracted_text = extract_text_from_pdf_usingfitz(chapiter_name)
print(extracted_text)

Not For Distribution, Sale or Reproduction.
401
11.1 PLAN RISK MANAGEMENT
Plan Risk Management is the process of deﬁning how to conduct risk management activities for a project. The key 
beneﬁt of this process is that it ensures that the degree, type, and visibility of risk management are proportionate to both 
risks and the importance of the project to the organization and other stakeholders. This process is performed once or 
at predeﬁned points in the project. The inputs, tools and techniques, and outputs of the process are depicted in Figure 
11-2. Figure 11-3 depicts the data ﬂow diagram for the process.
Figure 11-2. Plan Risk Management: Inputs, Tools & Techniques, and Outputs
Tools & Techniques
Inputs
Outputs
Plan Risk Management
.1 Expert judgment
.2 Data analysis
• Stakeholder analysis
.3 Meetings
.1 Project charter
.2 Project management plan
• All components
.3 Project documents
• Stakeholder register
.4 Enterprise environmental  
 
factors
.5 Organizational process assets
.1

In [4]:
import re 

regex_title=r"11(\.\d){1}\ .+"
regex_sub_title=r"11(\.\d){2}\ .+"
regex_sub2_title=r"11(\.\d){3}\ .+" # this for sub_sub_title example : 11.1.1.1
all_liste=[]
liste_titre=[]
liste_sub_title=[]
liste_sub2_title=[]
for regex in [regex_title,regex_sub_title,regex_sub2_title]:
    liste=[]
    pattern = re.compile(regex)

    matches=pattern.finditer(extracted_text)
    for match in matches :
        liste.append((match.group(),match.start()))
    all_liste.append([liste])
liste_titre=all_liste[0][0]
liste_sub_title=all_liste[1][0] 
liste_sub2_title =all_liste[2][0]  
all_liste=[]
print(liste_titre[6][0])

11.7 MONITOR RISKS


In [5]:
dic_title,dic_title_position=organaize_title_with_sub_title(liste_titre,liste_sub_title)

In [6]:
dic_sub_title,dic_sub_title_position=organaize_title_with_sub_title(liste_sub_title,liste_sub2_title)

In [7]:
dic_sub_title

{'11.1.1 PLAN RISK MANAGEMENT: INPUTS': ['11.1.1.1 PROJECT CHARTER',
  '11.1.1.2 PROJECT MANAGEMENT PLAN',
  '11.1.1.3 PROJECT DOCUMENTS',
  '11.1.1.4 ENTERPRISE ENVIRONMENTAL FACTORS',
  '11.1.1.5 ORGANIZATIONAL PROCESS ASSETS'],
 '11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQUES': ['11.1.2.1 EXPERT JUDGMENT',
  '11.1.2.2 DATA ANALYSIS',
  '11.1.2.3 MEETINGS'],
 '11.1.3 PLAN RISK MANAGEMENT: OUTPUTS': ['11.1.3.1 RISK MANAGEMENT PLAN'],
 '11.2.1 IDENTIFY RISKS: INPUTS': ['11.2.1.1 PROJECT MANAGEMENT PLAN',
  '11.2.1.2 PROJECT DOCUMENTS',
  '11.2.1.3 AGREEMENTS',
  '11.2.1.4 PROCUREMENT DOCUMENTATION',
  '11.2.1.5 ENTERPRISE ENVIRONMENTAL FACTORS',
  '11.2.1.6 ORGANIZATIONAL PROCESS ASSETS'],
 '11.2.2 IDENTIFY RISKS: TOOLS AND TECHNIQUES': ['11.2.2.1 EXPERT JUDGMENT',
  '11.2.2.2 DATA GATHERING',
  '11.2.2.3 DATA ANALYSIS',
  '11.2.2.4 INTERPERSONAL AND TEAM SKILLS',
  '11.2.2.5 PROMPT LISTS',
  '11.2.2.6 MEETINGS'],
 '11.2.3 IDENTIFY RISKS: OUTPUTS': ['11.2.3.1 RISK REGISTER',
  '11.2

In [8]:
dic_title

{'11.1 PLAN RISK MANAGEMENT': ['11.1.1 PLAN RISK MANAGEMENT: INPUTS',
  '11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQUES',
  '11.1.3 PLAN RISK MANAGEMENT: OUTPUTS'],
 '11.2 IDENTIFY RISKS': ['11.2.1 IDENTIFY RISKS: INPUTS',
  '11.2.2 IDENTIFY RISKS: TOOLS AND TECHNIQUES',
  '11.2.3 IDENTIFY RISKS: OUTPUTS'],
 '11.3 PERFORM QUALITATIVE RISK ANALYSIS': ['11.3.1 PERFORM QUALITATIVE RISK ANALYSIS: INPUTS',
  '11.3.2 PERFORM QUALITATIVE RISK ANALYSIS: TOOLS AND TECHNIQUES',
  '11.3.3 PERFORM QUALITATIVE RISK ANALYSIS: OUTPUTS'],
 '11.4 PERFORM QUANTITATIVE RISK ANALYSIS': ['11.4.1 PERFORM QUANTITATIVE RISK ANALYSIS: INPUTS',
  '11.4.2 PERFORM QUANTITATIVE RISK ANALYSIS: TOOLS AND TECHNIQUES',
  '11.4.3 PERFORM QUANTITATIVE RISK ANALYSIS: OUTPUTS'],
 '11.5 PLAN RISK RESPONSES': ['11.5.1 PLAN RISK RESPONSES: INPUTS',
  '11.5.2 PLAN RISK RESPONSES: TOOLS AND TECHNIQUES',
  '11.5.3 PLAN RISK RESPONSES: OUTPUTS'],
 '11.6 IMPLEMENT RISK RESPONSES': ['11.6.1 IMPLEMENT RISK RESPONSES: INPUTS',
  

In [9]:
# Extract_intro_title:
def Extract_intro_process(dic_title_position,text): # insert process dictionary 
    content_dic={}
    for key, value in dic_title_position.items():
        sorted_value=sort_liste_based_on_element(value,1)
        debut=key[1]
        end=sorted_value[0][1]
        content_dic[key[0]]=text[debut:end]
    return content_dic
    

In [10]:
description_process=Extract_intro_process(dic_title_position,extracted_text)

In [11]:
description_process.keys()

dict_keys(['11.1 PLAN RISK MANAGEMENT', '11.2 IDENTIFY RISKS', '11.3 PERFORM QUALITATIVE RISK ANALYSIS', '11.4 PERFORM QUANTITATIVE RISK ANALYSIS', '11.5 PLAN RISK RESPONSES', '11.6 IMPLEMENT RISK RESPONSES', '11.7 MONITOR RISKS'])

In [12]:
def Extract_Figure(text):
    regex=r'Figure \d+-\d+'
    pattern=re.compile(regex)
    liste=[]
    matchs=pattern.finditer(text)
    for match in matchs:
        liste.append(match.group())
    return set(liste)
def Extract_Table(text):
    regex=r'Table \d+-\d+'
    pattern=re.compile(regex)
    liste=[]
    matchs=pattern.finditer(text)
    for match in matchs:
        liste.append(match.group())
    return set(liste)

In [13]:
def clean_numeration(text,level=1):
    regex_title=r"11(\.\d){1}\s"
    regex_sub_title=r"11(\.\d){2}\s"
    regex_sub2_title=r"11(\.\d){3}\s" # this for sub_sub_title example : 11.1.1.1
    if level==1:
        regex=regex_title
        pattern = re.compile(regex)

        matches=pattern.finditer(text)
        for m in matches:
            return text.replace(m.group(),'')
    if level==2:
        regex=regex_sub_title
        
        pattern = re.compile(regex)

        matches=pattern.finditer(text)
        for m in matches:
            return text.replace(m.group(),'')
    if level==3:
        regex=regex_sub2_title
   
        pattern = re.compile(regex)

        matches=pattern.finditer(text)
        for m in matches:
            return text.replace(m.group(),'')   

In [14]:
clean_numeration('11.1.1.1 PLAN RISK MANAGEMENT',3)

'PLAN RISK MANAGEMENT'

In [15]:
clean_numeration('11.1.2 PLAN RISK MANAGEMENT TOOLS AND TECHNIQUES'.split(':')[0],2)

'PLAN RISK MANAGEMENT TOOLS AND TECHNIQUES'

In [16]:
def parent_process_relationship(dic_title,txt):
    for key,value in dic_title.items():
        if txt in value:
            parent,relationship=key.split(':')
            return clean_numeration(parent,2),relationship 

In [17]:
parent_process_relationship(dic_sub_title,'11.2.1.1 PROJECT MANAGEMENT PLAN')

('IDENTIFY RISKS', ' INPUTS')

In [18]:
import spacy

# Load English tokenizer, POS tagger, parser, and NER
nlp = spacy.load("en_core_web_sm")
import unicodedata

def normalize_text(text):
    return unicodedata.normalize('NFKC', text)


def string_in_liste(string,liste):
    statue=False
    i=0
    while (statue==False and i<len(liste)):
        
        if  liste[i] in normalize_text(string):
            statue=True
        else:
            i+=1
    return statue
        
def extract_actionable_items(text):
    doc = nlp(text)
    
    actionable_items = []
    
    for sent in doc.sents:
        if any(token.pos_ == "VERB" for token in sent):
            if sent[0].tag_ == "VB" or sent[0].tag_ == "VBG":
                
                actionable_items.append(sent.text)
    
    return actionable_items

def extract_definition(dictionary,text):
    doc = nlp(text)
    defintion=[]
    for sent in doc.sents:
        
        if any(normalize_text(token.lemma_) in dictionary for token in sent): 
            defintion.append(sent)
            break
            
    return defintion  
    
import re

def Extract_refenrence(text):
    regex=r'Described in Section.+\.\s'
    pattern=re.compile(regex)
    liste_ref=[]
    matchs=pattern.finditer(text)
    for match in matchs:
        liste_ref.append(match.group())
    return liste_ref    
# Extract Examples:
def Extract_example(text):
    example=[]
    regex=r'\b(for example|such as|include|as an example).*\.'
    pattern=re.compile(regex)
    
    matchs=pattern.finditer(text)
    for match in matchs :
        debut=match.span()[0]
        end=match.span()[1]
        sample=text[:debut]
        last_period_mark=sample[::-1].find('.')
        if last_period_mark!=-1:
            real_debut=debut-last_period_mark
        else:
            real_debut=debut
        example.append(text[real_debut:end])
    return example 

dic="define definition setting identify defining set establishing classifying specifying means meaning is"
doc = nlp(dic)
liste=[]
for token in doc:
    liste.append(str(token.lemma_))
definition_dic=list(set(liste))


def Extract_Usercases(text):
    Usercases=[]
  
    regex=r'\b(use case|scenario|application|situation).*\.'
    pattern=re.compile(regex)
    
    matchs=pattern.finditer(text)
    for match in matchs :
        debut=match.span()[0]
        end=match.span()[1]
        sample=text[:debut]
        last_period_mark=sample[::-1].find('.')
        if last_period_mark!=-1:
            real_debut=debut-last_period_mark
        else:
            real_debut=debut
        Usercases.append(text[real_debut:end])
    return Usercases         
    

C:\Users\Haboubi\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
import pandas as pd
dict_contenu=get_contenu(dic_title_position,extracted_text)
dataframe_contenu=pd.DataFrame(dict_contenu.items(),columns=['Process','Content'])

[[48, 12474], [12474, 34254], [34254, 51201], [51201, 72011], [72011, 100705], [100705, 108098], [108098, -1]]


In [20]:
dataframe_contenu

,Process,Content
0,11.1 PLAN RISK MANAGEMENT,11.1 PLAN RISK MANAGEMENT\nPlan Risk Managemen...
1,11.2 IDENTIFY RISKS,11.2 IDENTIFY RISKS\nIdentify Risks is the pro...
2,11.3 PERFORM QUALITATIVE RISK ANALYSIS,11.3 PERFORM QUALITATIVE RISK ANALYSIS\nPerfor...
3,11.4 PERFORM QUANTITATIVE RISK ANALYSIS,11.4 PERFORM QUANTITATIVE RISK ANALYSIS\nPerfo...
4,11.5 PLAN RISK RESPONSES,11.5 PLAN RISK RESPONSES\nPlan Risk Responses ...
5,11.6 IMPLEMENT RISK RESPONSES,11.6 IMPLEMENT RISK RESPONSES\nImplement Risk ...
6,11.7 MONITOR RISKS,11.7 MONITOR RISKS\nMonitor Risks is the proce...


In [21]:
dataframe_contenu.to_csv('process.csv',header=True,index=False)

In [22]:
dataframe_contenu['Intro']=dataframe_contenu['Process'].apply(lambda text: description_process[text])

In [23]:
dataframe_contenu['Examples']=dataframe_contenu['Intro'].apply(lambda txt:Extract_example(txt))
dataframe_contenu['Definitions']=dataframe_contenu['Intro'].apply(lambda txt:extract_definition(definition_dic,txt))
dataframe_contenu['Actionable_items']=dataframe_contenu['Intro'].apply(lambda txt:extract_actionable_items(txt))
dataframe_contenu['Extract_refenrence']=dataframe_contenu['Intro'].apply(lambda txt:Extract_refenrence(txt))
dataframe_contenu['UserCase']=dataframe_contenu['Intro'].apply(lambda txt:Extract_Usercases(txt))
dataframe_contenu['Figure']=dataframe_contenu['Intro'].apply(lambda txt:Extract_Figure(txt))
dataframe_contenu['Table']=dataframe_contenu['Intro'].apply(lambda txt:Extract_Table(txt))

In [24]:
dataframe_contenu

,Process,Content,Intro,Examples,Definitions,Actionable_items,Extract_refenrence,UserCase,Figure,Table
0,11.1 PLAN RISK MANAGEMENT,11.1 PLAN RISK MANAGEMENT\nPlan Risk Managemen...,11.1 PLAN RISK MANAGEMENT\nPlan Risk Managemen...,[],"[(11.1, PLAN, RISK, MANAGEMENT, \n, Plan, Risk...",[Figure 11-3 depicts the data ﬂow diagram for ...,[],[],"{Figure 11-2, Figure 11-3}",{}
1,11.2 IDENTIFY RISKS,11.2 IDENTIFY RISKS\nIdentify Risks is the pro...,11.2 IDENTIFY RISKS\nIdentify Risks is the pro...,[],"[(11.2, IDENTIFY, RISKS, \n, Identify, Risks, ...",[Figure 11-7 depicts the data ﬂow diagram for ...,[],[ The frequency of iteration and participation...,"{Figure 11-6, Figure 11-7}",{}
2,11.3 PERFORM QUALITATIVE RISK ANALYSIS,11.3 PERFORM QUALITATIVE RISK ANALYSIS\nPerfor...,11.3 PERFORM QUALITATIVE RISK ANALYSIS\nPerfor...,[],"[(11.3, PERFORM, QUALITATIVE, RISK, ANALYSIS, ...",[Figure 11-9 depicts the data ﬂow \ndiagram fo...,[],[],"{Figure 11-9, Figure 11-8}",{}
3,11.4 PERFORM QUANTITATIVE RISK ANALYSIS,11.4 PERFORM QUANTITATIVE RISK ANALYSIS\nPerfo...,11.4 PERFORM QUANTITATIVE RISK ANALYSIS\nPerfo...,[],"[(11.4, PERFORM, QUANTITATIVE, RISK, ANALYSIS,...","[Perform Quantitative Risk Analysis: Inputs, T...",[],[],"{Figure 11-12, Figure 11-11}",{}
4,11.5 PLAN RISK RESPONSES,11.5 PLAN RISK RESPONSES\nPlan Risk Responses ...,11.5 PLAN RISK RESPONSES\nPlan Risk Responses ...,"[ If developed, it may include identiﬁcation o...","[(11.5, PLAN, RISK, RESPONSES, \n, Plan, Risk,...","[Plan Risk Responses: Inputs, Tools & Techniqu...",[],[],"{Figure 11-16, Figure 11-17}",{}
5,11.6 IMPLEMENT RISK RESPONSES,11.6 IMPLEMENT RISK RESPONSES\nImplement Risk ...,11.6 IMPLEMENT RISK RESPONSES\nImplement Risk ...,[],"[(Implement, Risk, Responses, is, the, process...",[Figure 11-19 depicts the data ﬂow diagram for...,[],[],"{Figure 11-18, Figure 11-19}",{}
6,11.7 MONITOR RISKS,11.7 MONITOR RISKS\nMonitor Risks is the proce...,11.7 MONITOR RISKS\nMonitor Risks is the proce...,[],"[(11.7, MONITOR, RISKS, \n, Monitor, Risks, is...",[],[],[],"{Figure 11-21, Figure 11-20}",{}


In [26]:
dataframe_contenu.to_csv('big_process.csv',header=True,index=False)

In [25]:
import nltk
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Haboubi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Haboubi\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Haboubi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [26]:
tokens = nltk.word_tokenize(text)

# Perform NER
entities = nltk.ne_chunk(nltk.pos_tag(tokens))

# Display the named entities
for entity in entities:
    if hasattr(entity, 'label'):
        print(f'{entity.label()}: {" ".join(c[0] for c in entity.leaves())}')

NameError: name 'text' is not defined

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

# Sample sentence
def remove_stopwords(sentence):
    # Process the sentence with spaCy
    doc = nlp(sentence)
    # Filter out the stopwords
    filtered_words = [token.text for token in doc if token.text.lower() not in STOP_WORDS]
    return ' '.join(filtered_words)
# Process the sentence with spaCy

doc = nlp(phrase)

# Extract the main verb (in this simple example, assuming only one verb)
main_verb = next((token for token in doc if token.pos_ == 'VERB'), None)
print(main_verb)
if main_verb:
    print("Main verb:", main_verb.text)
    
    for child in main_verb.children:
        print(child.text)

In [ ]:
doc = nlp(phrase)

for token in doc :
    print(token.pos_,token.text)

In [ ]:
import unicodedata

def normalize_text(text):
    return unicodedata.normalize('NFKC', text)
def extract_key_words_influence_relationhsip(txt,relationship='influence',verb=True):
    ADP=False
    liste_word=[]
    word=''
    txt=normalize_text(txt)

    doc = nlp(txt)
    save_key_sentence=False
    still_word=False
    for token in doc :
        print(token.text,token.pos_)
        if( token.pos_ == 'VERB' and token.text=='influence'):
            ADP=True
        if ADP==True and token.pos_ == 'ADP':
            
            save_key_sentence=True
        if save_key_sentence==True or still_word==True:
            if token.pos_!='PRON' and token.pos_!='PUNCT ' :
                
                word=word+' '+token.text
                print(word)
            
            else:
                save_key_sentence==False
                liste_word.append(word)
                word=''
    if word !='':
        liste_word.append(word)
    return liste_word      


phrase = "11.1.1.4 ENTERPRISE ENVIRONMENTAL FACTORS The enterprise environmental factors that can inﬂuence the Plan Risk Management process include but are not limited to overall risk thresholds set by the organization or key stakeholders."
                
extract_key_words_influence_relationhsip(phrase)            
 
